# Transformer using PyTorch

Replicating the paper "Attention Is All You Need" from scratch using PyTorch.

Available here: https://arxiv.org/abs/1706.03762

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

import math
from pathlib import Path

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace



## Building blocks

### Input Embeddings
In NLP, words/tokens are represented as integers (token IDs), but since neural networks cannot work directly with numbers, these are first converted into continuous dense vectors via embeddings.

The embedding layer:
- Maps each token ID to a learnable vector of dimension `d_model`
- Allows the model to learn semantic meaning - similar words/tokens get similar embeddings during training

Refer to section 3.4 of the paper.

In [ ]:
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model) #maps token IDs to vectors

    def forward(self, x):
        """
        Args:
            x: Tensor of token indices, shape (batch_size, seq_len)

        Returns:
            Embedded tensor, scaled, shape (batch_size, seq_len, d_model)
        
        """
        return self.embedding(x) * math.sqrt(self.d_model)  #Refer section 3.4 of the paper (This is primarily done to balance the scale of the embeddings and positional encodings, and helps stabilize training early on.)


### Positional Encoding
Transformers process tokens in parallel, with no built-in sense of order. But language is sequential:

"The cat sat on the mat" =/= "The mat cat on the sat"

Positional information is injected into the input to create awareness of the token position, using Positional Encoding. They have the same dimension `d_model` as the input embeddings.

Refer to section 3.5 of the paper.

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        #Create matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        #Create a vector of shape (seq_len, 1)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) #tensor contains positions 0,1,...,seq_len - 1
        #Using section 3.5 to write out position encodings
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -math.log(10000.0) / d_model)

        pe[:, 0::2] = torch.sin(position * div_term) #sine to even positions
        pe[:, 1::2] = torch.sin(position * div_term) #cosine to odd positions

        #Add batch dimension to make the shape (1, seq_len, d_model), so it can be added to x during forward pass
        #with shape (batch_size, seq_len, d_model)
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe) #tensor gets saved with the model and moves with it to GPU/CPU

    def forward(self, x):
        x = x + (self.pe[:, :x.shape(1), :]).requires_grad_(False) #Add positional encodings to embeddings, requires_grad_() False means that this won't participate in backpropagation
        return self.dropout(x)
        

### Add + Norm Layer

- Layer Norm normalizes each sample’s hidden vector across dimensions, improving training dynamics and convergence.
- LayerNorm is preferred over BatchNorm because:
    1. It doesn't depend on batch statistics.
    2. Works well for variable-length sequences.

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, epsilon: float = 10**-6):
        super().__init__()
        self.epsilon = epsilon
        self.bias = nn.Parameter(torch.zeros(1)) #added (shift)
        self.alpha = nn.Parameter(torch.ones(1)) #multiplied (scale)

    def forward(self, x):
        mean = x.mean(dim = -1, keepdim = True)
        std = x.std(dim = -1, keepdim = True)

        return self.alpha * (x - mean) / (std + self.epsilon) + self.bias #From section


### Feed Forward layer

Discussed in section 3.3 of the paper, and equation 2

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff) #W1 and B1
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model) #W2 and B2

    def forward(self, x):
        return self.linear2(self.dropout(torch.relu(self.linear1(x)))) #(batch, seq_len, d_model) -> (batch, seq_len, d_ff) -> (batch, seq_len, d_model)

### Multi-Head Attention

Discussed in section 3.2.2 of the paper, this is the core innovation of the transformer model.
It is multiple scaled dot-product attention heads run in parallel, followed by a linear projection.

Refer to equation 1 for calculating attention scores, and then two equations in section 3.2.2 for multihead attention scores

Steps involved:
1. Project input `(Q, K, V)` into h subspaces using `W_q`, `W_k`, `W_v`.
2. Compute scaled dot-product attention for each head.
3. Concatenate all head outputs.
4. Pass through a final `W_o` projection layer.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, h, dropout): #h is the number of heads
        super().__init__()
        self.d_model = d_model #we have to ensure that d_model is divisible by h, d_model/h = d_k
        self.h = h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h

        self.w_q = nn.Linear(d_model, d_model) #W_q
        self.w_k = nn.Linear(d_model, d_model) #W_k
        self.w_v = nn.Linear(d_model, d_model) #W_v

        self.w_o = nn.Linear(d_model, d_model) #W_o
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1] #last dimension of Q,K,V

        # Calculate attention using Equation 1
        # (batch, h, seq_len, d_k) -> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k) #matrix multiplication of Q with K.transpose
        # Apply mask
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9) #values we don't want in the attention matrix will be replaced by a very low value
        
        # apply softmax
        attention_scores = attention_scores.softmax(dim = -1) #(batch, h, seq_len, seq_len)

        # apply dropout
        if dropout is not None:
            attention_scores = dropout(attention_scores)

        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q)     # (batch, seq_len, d_model) -> (batch, seq_len, d_model)
        key = self.w_k(k)       # (batch, seq_len, d_model) -> (batch, seq_len, d_model)
        value = self.w_v(v)     # (batch, seq_len, d_model) -> (batch, seq_len, d_model)
 
        # Reshaping for MultiHead Attention from [batch, seq_len, d_model] to [batch, seq_len, h, d_k]
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2) #(batch, seq_len, d_model) -> (batch, seq_len, h, d_k).transpose -> (batch, seq_len, d_model) -> (batch, h, seq_len, d_k)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2) #(batch, seq_len, d_model) -> (batch, seq_len, h, d_k).transpose -> (batch, seq_len, d_model) -> (batch, h, seq_len, d_k)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1,2) #(batch, seq_len, d_model) -> (batch, seq_len, h, d_k).transpose -> (batch, seq_len, d_model) -> (batch, h, seq_len, d_k)

        x, self.attention_scores = MultiHeadAttention.attention(query, key, value, mask, self.dropout)

        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k) #revert the transpose operation (batch, h, seq_len, d_k).transpose -> (batch, seq_len, d_model)
        
        #(batch, seq_len, d_model) -> (batch, seq_len, d_model)
        return self.w_o(x)

### Create skip connection (residual connection)

Addition is element-wise, and the result layer is normalized

`Output = LayerNorm(x + Sublayer(x))`

In [ ]:
class SkipConnection(nn.Module):
    def __init__(self, dropout):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNorm()

    #Applying Post-Norm here
    def forward(self, x, sublayer):
        return self.norm(x + self.dropout(sublayer(x)))

## Creating Encoder block
Creating the Encoder block, and stacking N of them together

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, 
                 self_attention_block: MultiHeadAttention, 
                 feed_forward_block: FeedForward,
                 dropout: float):
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([SkipConnection(dropout) for _ in range(2)])

    def forward(self, x, src_mask): #mask to hide padding words
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)

        return x


### Creating the Encoder object (N encoder blocks stacked)

Each block:
- Applies multi-head attention to every position with all other positions (self-attention)
- Applies a feedforward network
- Each sub-layer is wrapped with residual + layer norm

After all blocks, it applies a final layer norm.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNorm()

    def forward(self, x, mask):
        for layer in self.layers:  #Loop over each EncoderBlock in self.layers
            x = layer(x, mask)
        return self.norm(x)

## Creating Decoder Block

Uses the same classes as the Encoder block for the most part.
The key difference comes in with the Masked Multi-Head Attention (uses self-attention), and an additional, cross-attention Multi-Head Attention block.

Refer to section 3.1 in the paper


In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, 
                 self_attention_block: MultiHeadAttention,
                 cross_attention_block: MultiHeadAttention,
                 feed_forward_block: FeedForward,
                 dropout: float):
        super().__init__()

        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block

        self.residual_connections = nn.ModuleList([SkipConnection(dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask): #src_mask: encoder, tgt_mask: decoder
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask)) #self-attention block of decoder
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask)) #cross-attention takes in key from the decoder, and the query and value from the encoder, with the encoder mask
        x = self.residual_connections[2](x, self.feed_forward_block)

        return x


### Creating Decoder object (N decoder blocks stacked)



In [ ]:
class Decoder(nn.Module):
    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNorm()

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)

        return self.norm(x)
        


## Final Linear Layer
Projects the embeddings into a position in the vocabulary.

In [ ]:
class ProjectionLayer(nn.Module):
    def __init__(self,
                 d_model: int,
                 vocab_size: int):
        super().__init__()

        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        #(batch, seq_len, d_model) -> (batch, seq_len, vocab_size)

        return torch.log_softmax(self.proj(x), dim = -1)


## Transformer Block

We now have all the components we need to put together and create the Transformer block.

In [ ]:
class Transformer(nn.Module):
    def __init__(self,
                 encoder: Encoder,
                 decoder: Decoder,
                 src_embed: InputEmbeddings,
                 tgt_embed: InputEmbeddings,
                 src_pos: PositionalEncoding,
                 tgt_pos: PositionalEncoding,
                 projection_layer: ProjectionLayer
                 ):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    #define 3 methods: one to encode, one to decode, one to project

    def encode(self, src, src_mask):
        src = self.src_embed(src)   #apply embedding
        src = self.src_pos(src)     #apply positional encoding
        return self.encoder(src, src_mask)  #apply encoder block
    
    def decode(self, encoder_output, src_mask, tgt, tgt_mask):
        tgt = self.tgt_embed(tgt)   #apply embedding
        tgt = self.tgt_pos(tgt)     #apply positional encoding
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    
    def project(self, x):
        return self.projection_layer(x)
    

### Function to build transformer

Given all hyperparameters, this function builds the transformer and initializes the parameters with some initial values

In [ ]:
def build_transformer(src_vocab_size: int,
                      tgt_vocab_size: int,
                      src_seq_len: int,
                      tgt_seq_len: int,
                      d_model: int = 512,   #as mentioned in the paper
                      N: int = 6,           #number of encoder/decoder blocks
                      h: int = 8,           #number of heads
                      dropout: float = 0.1,
                      d_ff: int = 2048
) -> Transformer:
    
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    #create encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttention(d_model, h, dropout)
        feed_forward_block = FeedForward(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    #create decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttention(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttention(d_model, h, dropout)
        feed_forward_block = FeedForward(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(decoder_self_attention_block, 
                                     decoder_cross_attention_block,
                                     feed_forward_block,
                                     dropout)
        decoder_blocks.append(decoder_block)

    #Create the encoder and decoder
    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    #Create projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    #Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    #Initialize parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

# Creating English - Spanish translation Transformer

1. Using opus_books dataset from HuggingFace (Available here: https://huggingface.co/datasets/Helsinki-NLP/opus_books/viewer/en-es?views%5B%5D=en_es)
2. Use word-level tokenizer from HuggingFace - Splits sentence into tokens (Build vocabulary) (Documentation here: https://github.com/huggingface/tokenizers)

### Tokenizer

In [ ]:
def get_all_sentences(dataset, lang):
    for item in dataset:
        yield item['translation'][lang]

def get_or_build_tokenizer(config, dataset, lang):
    #config['tokenizer_file'] = '../tokenizers/tokenizer_{0}.json'
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        tokenizer = Tokenizer(WordLevel(unk_token='[UNK]')) #tokenizes unknown word
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "EOS"], min_frequency=2) #unknown word, padding, start of sentenece, end of sentence | min_frequency: for the word to appear in the vocabulary it needs to appear at least twice
        tokenizer.train_from_iterator(get_all_sentences(dataset, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))

    return tokenizer

## Dataset

### Custom Dataset Class

Use a custom Dataset class, following PyTorch documentation: https://docs.pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, dataset, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
        super().__init__()

        self.dataset = dataset
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang
        self.seq_len = seq_len

        #Save tokens to create tensors for the model: SOS, EOS and Padding
        #Use special method of tokenizer, token_to_id

        self.sos_token = torch.Tensor(tokenizer_src.token_to_id(['[SOS]']), dtype=torch.int64)
        self.eos_token = torch.Tensor(tokenizer_src.token_to_id(['[EOS]']), dtype=torch.int64)
        self.pad_token = torch.Tensor(tokenizer_src.token_to_id(['[PAD]']), dtype=torch.int64)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        src_target_pair = self.dataset[index]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]

        enc_input_tokens = self.tokenizer_src.encode(src_text).ids #input ids as array
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

        #padding tokens used to match seq_len
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2 #2 for the SOS and EOS tokens
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1 #Decoder only has SOS, and target only has EOS

        #make sure padding token length is sufficient/non-negative
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError('Sentence is too long')
        

        #one sentence goes to encoder input, one goes to decoder input, one output sentence
        #Add SOS and EOS to source text
        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(enc_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64)
                ]
        )

        decoder_input = torch.cat(  #no EOS here, only SOS
            [
                self.sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
            ]
        )

        decoder_output = torch.cat(  #add EOS to decoder output
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
            ]
        )

        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert decoder_output.size(0) == self.seq_len

        return {
            "encoder_input": encoder_input, #(seq_len)
            "decoder_input": decoder_input, #(seq_len)
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(), # (1, 1, seq_len) | mask: these tokens should not be seen by the attention mechanism
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & causal_mask(decoder_input.size(0)), # (1, seq_len) & (1, seq_len, seq_len) | causal mask, so that the decoder doesn't look at next position inputs
            "decoder_output": decoder_output, # (seq_len)
            "src_text": src_text,
            "tgt_text": tgt_text
        }

def causal_mask(size): #so that the decoder doesn't look at next position inputs
    mask = torch.triu(torch.ones(1, size, size), diagonal = 1).type(torch.int) #grabs upper triangular matrix, and makes it 0 | upper triangular matrix shows next position inputs
    return mask == 0

### Create dataset

- Download dataset 
- Create test dataset, val dataset, 
- Send to DataLoader to be used in training loop

In [ ]:
def get_dataset(config):
    dataset_raw = load_dataset('Helsinki-NLP/opus_books', f'{config["lang_src"]} - {config["lang_tgt"]}', split='train') #dataset and subset, ex: en-es

    #build tokenizer
    tokenizer_src = get_or_build_tokenizer(config=config, 
                                           dataset=dataset_raw,
                                           lang = config["lang_src"])
    tokenizer_tgt = get_or_build_tokenizer(config=config, 
                                           dataset=dataset_raw,
                                           lang = config["lang_tgt"])
    
    #making custom split for training and validation (90-10)
    train_dataset_size = int(0.9 * len(dataset_raw))
    val_dataset_size = len(dataset_raw) - train_dataset_size
    train_dataset_raw, val_dataset_raw = random_split(dataset_raw, [train_dataset_size, val_dataset_size])

    train_dataset = TranslationDataset(dataset=train_dataset_raw, 
                                       tokenizer_src=tokenizer_src,
                                       tokenizer_tgt=tokenizer_tgt,
                                       src_lang=config['lang_src'],
                                       tgt_lang=config['lang_tgt'],
                                       seq_len=config['seq_len'])
    
    val_dataset = TranslationDataset(dataset=val_dataset_raw, 
                                       tokenizer_src=tokenizer_src,
                                       tokenizer_tgt=tokenizer_tgt,
                                       src_lang=config['lang_src'],
                                       tgt_lang=config['lang_tgt'],
                                       seq_len=config['seq_len'])
    
    #setting max_len based on the dataset src and tgt lengths
    max_len_src, max_len_tgt = 0, 0

    for item in dataset_raw:
        src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_src.encode(item['translation'][config['lang_tgt']]).ids

        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

    print(f"Max length of source sentence: {max_len_src}")
    print(f"Max length of target sentence: {max_len_tgt}")

    #Create DataLoaders

    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True) #batch size 1 to process each sentence one by one

    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

### Build model

Based on vocab_size, it builds the transformer model